<a href="https://colab.research.google.com/github/gkrisp98/EMNIST-Classifier/blob/main/EMNIST_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt


In [3]:
#LOAD DATA
data = torchvision.datasets.EMNIST(root = './data/EMNIST', split = 'digits', download = True, train = True, 
                                   transform=transforms.Compose([transforms.ToTensor()]))

  0%|          | 0/561753746 [00:00<?, ?it/s]

Extracting ./data/EMNIST/EMNIST/raw/gzip.zip to ./data/EMNIST/EMNIST/raw


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [18]:
#SPLIT TRAINING AND VALIDATION SET

train_size = int(0.8*len(data))
valid_size = int(0.2*len(data))

train_set, valid_set = random_split(data, [train_size, valid_size])

In [31]:
#NEURAL NETWORK MODEL
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3)
    
    self.fc1 = nn.Linear(in_features=4*4*16, out_features=500)
    self.out = nn.Linear(in_features=500, out_features=10)

  def forward(self, t):
    #hidden conv layer 1, channel=6
    t = self.conv1(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size=2, stride=2)

    #hidden conv layer 2, channel=16
    t = self.conv2(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size=2, stride=3)

    #hidden linear layer 1
    t = t.reshape(-1,4*4*16)
    t = self.fc1(t)
    t = F.relu(t)

    t = self.out(t)
    t = F.softmax(t, dim=1)

    return t

In [24]:
#TRAIN LOADER
train_loader = DataLoader(train_set, batch_size=128, shuffle = True)
train_data = next(iter(train_loader))
#VALID LOADER
valid_loader = DataLoader(valid_set, batch_size = 64, shuffle = True)
valid_data = next(iter(valid_loader))

In [25]:
def get_num_correct(preds,labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [27]:
#TRAINING AND VALIDATION LOOP
model = Network()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

from torch.utils.tensorboard import SummaryWriter
torch.set_printoptions(linewidth=120)
writer = SummaryWriter()

for epoch in range(10):

  train_loss = 0
  train_correct = 0
  model.train()
  for batch in train_loader:
    images, labels = batch

    preds = model(images)
    loss = F.cross_entropy(preds, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    train_correct += get_num_correct(preds, labels)
    writer.add_scalar('Train Loss', train_loss, epoch)
    writer.add_scalar('Train Accuracy', train_correct/len(train_set), epoch)

  valid_loss = 0
  valid_correct = 0
  model.eval()
  for batch in valid_loader:
    images, labels = batch
    preds = model(images)
    loss = F.cross_entropy(preds, labels)

    valid_loss += loss.item()
    valid_correct += get_num_correct(preds, labels)
    writer.add_scalar('Valid Loss', valid_loss, epoch)
    writer.add_scalar('Validation Accuracy', valid_correct/len(valid_set), epoch)

  print("epoch:", epoch, "training loss:", train_loss, "training correct:", train_correct
        , "valid loss:", valid_loss, "valid correct", valid_correct)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch: 0 training loss: 3452.5621967315674 training correct: 34733 valid loss: 1725.575014591217 valid correct 11769
epoch: 1 training loss: 3447.652743577957 training correct: 73099 valid loss: 1720.5486121177673 valid correct 20840
epoch: 2 training loss: 3254.1845713853836 training correct: 76655 valid loss: 1289.172939300537 valid correct 39307
epoch: 3 training loss: 2422.624045610428 training correct: 168437 valid loss: 1182.513825893402 valid correct 43233
epoch: 4 training loss: 2342.0604940652847 training correct: 175260 valid loss: 1164.1532714366913 valid correct 44109
epoch: 5 training loss: 2314.455839395523 training correct: 178004 valid loss: 1153.6060922145844 valid correct 44697
epoch: 6 training loss: 2297.237192749977 training correct: 179830 valid loss: 1145.3629208803177 valid correct 45172
epoch: 7 training loss: 2284.729422688484 training correct: 181250 valid loss: 1140.0101540088654 valid correct 45443
epoch: 8 training loss: 2275.1303157806396 training correct

In [28]:
#TRAINING ACCURACY 
train_correct/len(train_set)

0.954203125

In [29]:
#VALIDATION ACCURACY
valid_correct/len(valid_set)

0.9537916666666667

In [ ]:
#PRINT ON TENSORBOARD
!kill 102
%load_ext tensorboard
%tensorboard --logdir=runs

In [36]:
#TEST DATA & TEST LOADER
test_data = torchvision.datasets.EMNIST(root = './data/EMNIST', split = 'digits', download = True, train = False, 
                                   transform=transforms.Compose([transforms.ToTensor()]))
test_loader = DataLoader(test_data, batch_size = 64, shuffle = True)
test_data = next(iter(test_loader))

In [37]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # calculate outputs by running images through the network
        preds = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(preds.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy: %d %%' % (
    100 * correct / total))

Accuracy: 95 %
